In [15]:
import joblib
from sklearn.svm import SVC
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from skbold.utils import MvpResultsClassification
import numpy as np
import nibabel as nib
from sklearn.cross_validation import StratifiedKFold

In [2]:
mvp = joblib.load('/media/lukas/data/DecodingEmotions/DATA/DATA_MVPA/Validation_set/glm_SELF/sub002/mvp.jl')
mvp.featureset_id -= 1

In [22]:
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('ufs', SelectKBest(f_classif, k=10000)),
        ('svm', SVC(kernel='linear', decision_function_shape='ovo'))
    ])
mvpres = MvpResultsClassification(mvp=mvp, n_iter=20, out_path=None, feature_scoring='fwm')

In [23]:
for train_idx, test_idx in StratifiedKFold(mvp.y, n_folds=20):
    
    train, test = mvp.X[train_idx, :], mvp.X[test_idx, :]
    y_train, y_test = mvp.y[train_idx], mvp.y[test_idx]
    
    pipe.fit(train, y_train)
    pred = pipe.predict(test)
    mvpres.update(test_idx, pred, pipe)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [24]:
mvpres.compute_scores()
mvpres.write(multiclass='ovo')



      Accuracy        F1  Precision   Recall  n_voxels
mean   0.65000  0.616667   0.650000  0.65000   10000.0
std    0.14204  0.157280   0.209505  0.14204       0.0

Confusion matrix:
[[ 30.   5.   5.]
 [  8.  24.   8.]
 [  7.   9.  24.]]


0
[0]


In [ ]:
from itertools import chain, combinations
from scipy.misc import comb
n, k = 3, 2
n_models = comb(n, k, exact=True)
cmb = list(combinations(range(n_models), 2))

In [ ]:
scores = np.zeros((coef.shape[1], 3))

for number in range(n_models):
    
    print('model nr %i\n---------\n' % number)
    
    for i, c in enumerate(cmb):
        
        print('combination: %r' % list(c))
        
        if number in c:
            print('Model %i in %r' % (number, list(c)))
            print('index: %i' % c.index(number))

            if c.index(number) == 1:
                print('reversing')
                val = coef[i, :] * -1
            else:
                val = coef[i, :]
            
            scores[:, number] += val
        print('\n')
scores = scores / 3

In [ ]:
np.sum(scores[:, 0] == 0).sum()

In [ ]:
wb = np.zeros(mvp.data_shape).ravel()
#coef_tmp = np.zeros(mvp.X.shape[1])
#coef_tmp[idx] = coef[2, :]
wb[mvp.voxel_idx] = scores[:, 2]
wb = wb.reshape(mvp.data_shape)
img = nib.Nifti1Image(wb, affine=mvp.affine)
img.to_filename('/home/lukas/testtest3')